<a 
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling 
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (Take note that you will not be implementing the encoder part of this tutorial.)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [157]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('/content/Star Wars Episode IV.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2022-02-13 08:39:54--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 3.221.126.233, 54.156.235.193, 18.214.211.171, ...
Connecting to piazza.com (piazza.com)|3.221.126.233|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2022-02-13 08:39:55--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 13.249.137.78, 13.249.137.97, 13.249.137.61, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|13.249.137.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  --.-KB

In [166]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  
print(random_chunk())

of this battle station."
"286" "TAGGE" "And what of the Rebellion?  If the Rebels have obtained a complete technical readout of this station, it is possible, however unlikely, that they might find a we


In [165]:
import torch
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


---

## Part 4: Creating your own GRU cell 

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please try not to look at the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**
* Create a custom GRU cell

**DONE:**



In [164]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.gru_cells = nn.ModuleList([])

    for l in range(self.num_layers):
      if l == 0:
        gru_size = self.input_size
      else:
        gru_size = self.hidden_size
      self.gru_cells.append(GRUCell(gru_size, hidden_size))
  
  def forward(self, inputs, hidden):
    outputs, hiddens = self.gru_cells[0](inputs, hidden[0:1])

    for l in range(1, self.num_layers):
      outputs, hidden_layer = self.gru_cells[l](outputs, hidden[l:l+1])
      hiddens = torch.cat((hiddens, hidden_layer), dim=0)

    return outputs, hiddens  

class GRUCell(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(GRUCell, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    
    self.sigmo = nn.Sigmoid()
    self.tanh = nn.Tanh()
    self.W_r = nn.Linear(input_size + hidden_size, hidden_size)
    self.W_z = nn.Linear(input_size + hidden_size, hidden_size)
    self.W = nn.Linear(input_size + hidden_size, hidden_size)
    
  def forward(self, input, prev_hidden):
    z_t = self.sigmo(self.W_z(torch.cat((input, prev_hidden), dim=2)))
    r_t = self.sigmo(self.W_r(torch.cat((input, prev_hidden), dim=2)))
    
    h_t = self.tanh(self.W(torch.cat((torch.mul(r_t, prev_hidden), input), dim=2)))
    h = torch.mul((1 - z_t), prev_hidden) + torch.mul(z_t, h_t)
    
    output = h
    hidden = h
    return output, h



---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**
* Create an RNN class that extends from nn.Module.

**DONE:**



In [163]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    
    # more stuff here...
    self.embedding = nn.Embedding(self.input_size, self.hidden_size)
    self.gru = GRU(self.hidden_size, self.hidden_size, self.n_layers)
    self.out = nn.Linear(self.hidden_size, self.output_size)
    self.relu = nn.ReLU()
    

  def forward(self, input_char, hidden_state):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU

    # stuff here
    embedded = self.embedding(input_char).view(1, 1, -1)
    output, hidden = self.gru(embedded, hidden_state)
    output = self.relu(self.out(output))

    return output, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [162]:
def random_training_set():    
  chunk = random_chunk()
  # print(chunk)
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes. 

**TODO:**
* Fill in the pieces.

**DONE:**




In [161]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target):
  ## initialize hidden layers, set up gradient and loss 
    # your code here
  ## /

  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  loss = 0

  for c in range(chunk_len):
    c_decoded, hidden = decoder(inp[c], hidden)
    loss += criterion(c_decoded.squeeze(0), target[c].unsqueeze(0))
  
  loss.backward()
  decoder_optimizer.step()
  loss_score = loss.item() / chunk_len

  return loss_score

---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**
* Fill out the evaluate function to generate text frome a primed string

**DONE:**



In [153]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  ## initialize hidden state, initialize other useful variables
    # your code here
  ## /
  hidden = decoder.init_hidden()
  predicted = prime_str
  prime_characters = char_tensor(prime_str)

  for i in range(len(prime_str) - 1):
    _, hidden = decoder(prime_characters[i], hidden)
  
  eval_character = prime_characters[-1]

  for j in range(predict_len):
    out, hidden = decoder(eval_character, hidden)
    sample = sample_outputs(out.data.view(-1), temperature)

    choice = all_characters[sample]
    eval_character = char_tensor(choice)
    predicted += choice

    # pdb.set_trace()
  
  return predicted
  

---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---


**TODO:** 
* Create some cool output

**DONE:**




Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs. These are the results, along with the prime string:

---

 G:
 
 Gandalf was decrond. 
'All have lord you. Forward the road at least walk this is stuff, and 
went to the long grey housel-winding and kindled side was a sleep pleasuring, I do long 
row hrough. In  

 lo:
 
 lost death it. 
'The last of the gatherings and take you,' said Aragorn, shining out of the Gate. 
'Yes, as you there were remembaused to seen their pass, when? What 
said here, such seven an the sear 

 lo:
 
 low, and frod to keepn 
Came of their most. But here priced doubtless to an Sam up is 
masters; he left hor as they are looked. And he could now the long to stout in the right fro horseless of 
the like 

 I:
 
 I had been the 
in his eyes with the perushed to lest, if then only the ring and the legended 
of the less of the long they which as the 
enders of Orcovered and smood, and the p 

 I:
 
 I they were not the lord of the hoomes. 
Home already well from the Elves. And he sat strength, and we 
housed out of the good of the days to the mountains from his perith. 

'Yess! Where though as if  

 Th:
 
 There yarden 
you would guard the hoor might. Far and then may was 
croties, too began to see the drumbred many line 
and was then hoard walk and they heart, and the chair of the 
Ents of way, might was 

 G:
 
 Gandalf 
been lat of less the round of the stump; both and seemed to the trees and perished they 
lay are speered the less; and the wind the steep and have to she 
precious. There was in the oonly went 

 wh:
 
 which went out of the door. 
Hull the King and of the The days of his brodo 
stumbler of the windard was a thing there, then it been shining langing 
to him poor land. They hands; though they seemed ou 

 ra:
 
 rather,' have all the least deather 
down of the truven beginning to the house of sunk. 
'Nark shorts of the Eyes of the Gate your great nothing as Eret. 
'I wander trust horn, and there were not, it  

 I:
 
 I can have no mind 
together! Where don't may had one may little blung 
terrible to tales. And turn and Gandalf shall be not to as only the Cattring 
not stopped great the out them forms. On they she lo 

---


In [154]:
import time
n_epochs = 5000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 2
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

In [155]:
# n_epochs = 2000
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[111.60166549682617 (200 4%) 2.4159]
Whe the us, and *abours to saritde the winked Zramy themres andt on Xlentor softhe int lome wand ot, a 

[220.24724102020264 (400 8%) 2.1502]
Whsund reping ay ay the roth the care some dides, she one lyon to ow and eren, and or terene ase the  

[327.6690442562103 (600 12%) 1.8674]
Whing a down was and stall the you suclledy or of shook not suspung the looked at minse, and 








 

[435.39996695518494 (800 16%) 1.9827]
Whed, would were, camled gonney ony agaar move ahove other undell the word his word the down sighFrost 

[556.5108437538147 (1000 20%) 1.7561]
Whing if that was cag!' saiked who have make on thim, and not should got on then his shive his truits  

[665.9958119392395 (1200 24%) 1.6033]
Whien this what &ro`e, his 
look, Lut and all the reast too a father 
had shall then the swars 
hon(r  

[774.2663886547089 (1400 28%) 1.6684]
Wh 
men than ever to the rest still gainly dark and still came acried all from 
grew and rearor his fe 

[8

In [156]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

 he
 he were 
step and look and pace glad tider on 
come to 
try to you wise, and 








were goes that watcher stew a huddle for no at last afraithless on 
and {oung on fance, but drove the companished li 

 ca
 carry and come to me stars and courself, and when you 
did not all the forest grace in the grass of their deepening of the sgones won't some at him was slow and died days of the 
would not the fail}y of 

 Th
 Thjoden 
that is 
more men ceon?' 

'That's the trees are 
in the passed, not lookly so Thjoden to the doorKhere wind one better to mmuch many closed knows of my charp had strange 
came, do you knows pl 

 ra
 raised down to get down down, 'the 








now to shall be a prumbly to here, and 
trust at the =road even 
as the days, when the in 
the 




more his 
way and now what was nearly him to do to 
covers 

 G
 Gollum 
so the lights, but if clad and 
your part 




now more on the 
end of when the traith or 
anly when they said, which things to the 
started, alos o

---

## Part 6: Generate output on a different dataset

---

**TODO:**

* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle 

* Generate some decent looking results and evaluate your model's performance (say what it did well / not so well)

**DONE:**



In [167]:
# Training on the Script from Star Wars Episode IV: A New Hope
import time
n_epochs = 5000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 2
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[112.06821966171265 (200 4%) 2.3851]
Whc ell =ePeodert Re hgo lofer."
"222" "cUent."
"461" "Pdene hen, hepe sugis seiytou."
"12" "jnt Fe 

[223.20258116722107 (400 8%) 2.4407]
Where tus st?  tel trout in dnoA"
"27" "I>E" "Moand this in."
  HeloU ree plond denthe 

[357.6770758628845 (600 12%) 2.0028]
WhjMD sightes.  ! Jipe."
's c8ire is the king he well!   on't I droing 'isse stri puster."
"8 

[501.9059684276581 (800 16%) 1.9428]
Whe giction the could the do(+You going in ;ere...  I stO-I
"3J" "OA\Rs I seed We'd m-The to Qoing in  

[643.82537150383 (1000 20%) 1.7886]
Wh the cleSt 0>[en here..."
"565" "HAN" "He st8ick it!  Kid!"
"89" "THREEPIO" "I for sir."
"261" "yATh 

[786.3125822544098 (1200 24%) 1.5581]
Wh" "zeter he c^rest it 1332" "THREEPIO" "I c6Mriden trince.  I'll I's luite or Bust You on't kid he l 

[929.8766872882843 (1400 28%) 1.7641]
orse the to sTe}vount."
"850" "THREEPIO" "The 1:4" "CO%ceDetrou  

[1072.8273215293884 (1600 32%) 1.6606]
Whole st^ng here."
"518" "THR

In [168]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

 he
 help.  I don't have to contrugh more for sad so..."
"204" "THREEPIO" "I want you to make they must also all the planet good overative starships.  Flast the end here."
"555" "HAN" "Not an eyes there..."
 

 he
 help move?"
"359" "fEATARKIN" "This is some space still, sir.  I can't see and were been short-citching to let a starships?  What are you ain't amaty fire the galayong to find your father?"
"450" ">AN"  

 ca
 can do bot one of this technolow that they idenctionaly way or against the money only one-man mantoor your run."
"994" "REm one-man fire yourself of anything... that the battle station."
"426" "HAN" "Ch 

 lo
 lost standing by."
"547" "HAN" "It's power of the plans to that way!  Memnet this, but do you think you let you say sir, I think she would I need you the door.  I wonder of there was a whole droids.  It 

 ca
 can't get out of here enough.  In quite for sure, sir, no little from the plans that."
"955" "RE+" "It's a lottached to talking."
"863" "REven if I can't m

Because of the format of the text document: "LineNumber" "Name": Line
I think my model had more trouble than with the Lord of the Rings text. There are not only numbers, but also symbols in the names and lines that were not there in the Lord of the Rings text, so it had a harder time making sense of them. However, it started to recognize where the symbols belonged through training, and generally put them in the right place after fully training. I think that this could have been fixed if I had extended the length of the passage taken in to around 2x the amount of characters used by the Lord of the Rings training. It was interesting to see the results even still. The model definitely picked up a lot of the most common phrases and words, like "Force", "System", "ship", "Red Three", and "rebels". If I had more time, I would train the model again with twice the amount of characters so it could take in more lines at a time.

By the end, the model successfully created some lines that convincingly sound like they're out of a Star Wars movie! It even matched unique ways of speaking with the correct characters, like C3PO, Grand Moff Tarkin, and Han Solo. With more training, I'm sure that the model would do even better.